In [30]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
import pathlib
import pickle #to save files
from itertools import product
from scipy.stats import skew, kurtosis, pearsonr
from scipy.signal import butter, welch, filtfilt
import nolds

# from PreprocessFcns import gen_clips, powerspectra

%matplotlib inline
pd.set_option('display.max_rows', None)

In [15]:
datapath = r'D:\CIS-PD Study\Subjects'
timepath = r'C:\Users\Alex\Google Drive\Shared\SRALAB\PD'
file = 'PD_timestampCorrections.csv'

activities = list(['Heart Rate Variability', 'MDS-UPDRS #1: Finger Tapping',
           'MDS-UPDRS #2: Hand Movements', 'MDS-UPDRS #3: Pronation-Supination',
           'MDS-UPDRS #4: Toe Tapping', 'MDS-UPDRS #5: Leg Agility',
           'MDS-UPDRS #6: Arising from Chair', 'MDS-UPDRS #7: Gait',
           'MDS-UPDRS #8: Postural Stability', 'MDS-UPDRS #9: Postural Hand Tremor',
           'MDS-UPDRS #10: Kinetic Hand Tremor', 'MDS-UPDRS #11: Rest Tremor',
           'Motor #1: Standing', 'Motor #2: Walking', 'Motor #3: Walking while Counting',
           'Motor #4: Finger to Nose', 'Motor #5: Alternating Hand Movements',
           'Motor #6: Sit to Stand', 'Motor #7: Drawing on Paper',
           'Motor #8: Typing on a Computer', 'Motor #9: Nuts and Bolts',
           'Motor #10: Drinking Water', 'Motor #11: Organizing Folder',
           'Motor #12: Folding Towels', 'Motor #13: Sitting'])



In [8]:
def process_annotations(path):
#---------------------------------------------------------------------------------------------------------
# Processes raw annotations file to extract start / end timestamps and remove unnecessary data
#
# Inputs:  path - filepath of the subject folder containing annotations.csv
#
# Outputs: df - dataframe containing list of activities and their start / end timestamps
#---------------------------------------------------------------------------------------------------------
    df = pd.read_csv(os.path.join(path, 'annotations.csv'))
    del df['Timestamp (ms)']
    del df['AnnotationId']
    del df['AuthorId']
    del df['Value']
    
    df = df[(df.EventType != 'Testing Day')]
    
    sorter = set(df.EventType.unique().flatten())
    sorterIndex = dict(zip(sorter, range(len(sorter))))
        
    df['EventType_Rank'] = df['EventType'].map(sorterIndex)
    df['Cycle'] = df.groupby('EventType')['Start Timestamp (ms)'].rank(ascending=True).astype(int)
    del df['EventType_Rank']
    df[df['EventType'].str.contains('Heart')] = df[df['EventType'].str.contains('Heart')].replace(to_replace={'Cycle': {1: 'NaN', 2: 'NaN', 3: 'NaN', 4: 'NaN'}})
    df = df.reset_index(drop=True).set_index('EventType')
    
    return df

In [204]:
timepath2 = os.path.join(timepath,file)
timestamps = pd.read_csv(timepath2)
participants = list(timestamps.iloc[15])
participants = [ int(x) for x in participants[1:]]


,Motor #4: Finger to Nose,1004,1016,1018,1019,1020,1024,1029,1030,1032
1,Start_1,5.0,4.0,0.0,1.0,3.0,-2.0,4.0,3.0,5.0
2,End_1,5.0,6.0,5.0,4.0,4.0,4.0,5.0,4.0,6.0
3,Start_2,4.0,4.0,3.0,3.0,3.0,5.0,1.0,3.0,3.0
4,End_2,5.0,5.0,4.0,4.0,5.0,6.0,4.0,5.0,5.0
5,Start_3,5.0,7.0,3.0,4.0,1.0,1.0,1.0,2.0,4.0
6,End_3,4.0,4.0,5.0,6.0,4.0,5.0,4.0,6.0,5.0
7,Start_4,4.0,4.0,3.0,5.0,4.0,3.0,2.0,0.0,4.0
8,End_4,5.0,4.0,4.0,4.0,5.0,9.0,4.0,6.0,6.0
9,Start_5,4.0,5.0,4.0,3.0,2.0,0.0,4.0,3.0,4.0
10,End_5,5.0,4.0,5.0,4.0,4.0,5.0,4.0,4.0,5.0


In [194]:
participant = 1004
path = os.path.join(datapath,str(participant))
ot = process_annotations(path)

motor6 = ot.loc['Motor #6: Sit to Stand']
motor2 = ot.loc['Motor #2: Walking']
motor8 = ot.loc['Motor #8: Typing on a Computer']
motor4 = ot.loc['Motor #4: Finger to Nose']
motor6

,Start Timestamp (ms),Stop Timestamp (ms),Cycle
EventType,,,
Motor #6: Sit to Stand,1499435123853,1499435132990,1
Motor #6: Sit to Stand,1499437789041,1499437800350,2
Motor #6: Sit to Stand,1499440170569,1499440179986,3
Motor #6: Sit to Stand,1499442106922,1499442115139,4
Motor #6: Sit to Stand,1499443589910,1499443598771,5
Motor #6: Sit to Stand,1499445064410,1499445074523,6
Motor #6: Sit to Stand,1500386245726,1500386256353,7


In [236]:
participant = 1004

def adjust_timestamps(participant,number,datapath,timestamps):
## Participant -- 4 digit participant number
## motor -------- Number 2,4,6, or 8 corresponding to desired motor assessment
## datapath ----- Path to Subjects folder
## ts ----------- Timestamps dataframe for desired motor assessment (eg. ts6)
##
## Output, m ---- Dataframe with correct timestamps
    
    path = os.path.join(datapath,str(participant))
    ot = process_annotations(path)
    if number == 2:
        m = ot.loc['Motor #2: Walking']
        ts = timestamps.iloc[15:30]
    elif number == 4:
        m = ot.loc['Motor #4: Finger to Nose']
        ts = timestamps.iloc[47:]
    elif number == 6:
        m = ot.loc['Motor #6: Sit to Stand']
        ts = timestamps.iloc[:14]
    elif number == 8:
        m = ot.loc['Motor #8: Typing on a Computer']
        ts = timestamps.iloc[31:46]
        
    old = ts.columns[0]
    new = ts.iloc[0][0]
    ts = ts.rename(columns={old:new})
    ts.reset_index(inplace=True)
    del ts['index']
    ts.drop(0, inplace=True)
    
    times = ts[str(participant)]
    m.reset_index(inplace=True)

    for x in range(0,int(len(times))):
        if x % 2 == 0:
            y = int(x/2)
            print(x)
            old = int(m.loc[y]['Start Timestamp (ms)'])
            print(old,'\n',type(old),'\n')
            print(times[x])
            #new = old + int(times[x])*1000
            #m.set_value(y,'Start Timestamp (ms)',new)
        else:
            y = int((x-1)/2)
            old = int(m.loc[y]['Stop Timestamp (ms)'])
            new = int(old) + int(times[x])*1000
            m.set_value(y,'Stop Timestamp (ms)',new)
    return m

In [237]:
t = adjust_timestamps(1004,6,datapath,timestamps)
t

0
1499435123853 
 <class 'int'> 



KeyError: 0

In [233]:
int(times[2])*1000

0